In [1]:
#r "nuget:Microsoft.ML"
    
//Install Daany packages
#r "nuget:Daany.DataFrame"
#r "nuget:Daany.DataFrame.Ext"
#r "nuget:Daany.Stat"
    
//Install XPlot package
#r "nuget:XPlot.Plotly"

Installing package Microsoft.ML................................................done!

Successfully added reference to package Microsoft.ML, version 1.4.0

Installing package Daany.DataFrame...............done!

Successfully added reference to package Daany.DataFrame, version 0.6.0

Installing package Daany.DataFrame.Ext............done!

Successfully added reference to package Daany.DataFrame.Ext, version 0.6.0

Installing package Daany.Stat...............done!

Successfully added reference to package Daany.Stat, version 0.6.0

Installing package XPlot.Plotly...............done!

Successfully added reference to package XPlot.Plotly, version 3.0.1

# Predictive Maintenance on .NET Platform

## Summary
This article is based on the Azure AI Gallery article: [Predictive Maintenance Modelling Guide]( https://gallery.azure.ai/Collection/Predictive-Maintenance-Modelling-Guide-1). The datasets used in the article can also be found at Azure AI Gallery. 

However, this notebook is completely implemented on .NET platform using `C# Jupyter Notebook` and `Daany` - C# data analytics library. There are small differences between this notebook and the notebooks at the official azure gallery portal, but in most cases, the code follows the steps defined there. 
The purpose of this notebook is to demonstrate how to use `.NET Jupyter Notebook` with `Daany.DataFrame` and `ML.NET` in order to prepare the data and build the Predictive Maintenance Model on .NET platform.  

## Quick Introduction to Predictive Maintenance

Simply speaking it is a technique to determine (predict) the failure of the machine component in the near future so that the component can be replaced programmatically based on the maintenance plane before it fails and stop the production process. In order to handle such a situation can improve the production process and increase productivity. With successfully handling with predictive maintenance we are able to achieve the following goals:

- reduce the operational risk of mission-critical equipment
- control cost of maintenance by enabling just-in-time maintenance operations
- discover patterns connected to various maintenance problems
- provide Key Performance Indicators.

The following image shows diferent type of maintenance in the production.

![predictive maintenance diagram](img/predictive_maintenance01.png)

### Predictive maintenance data collection

In order to handle and use this tecnique we we need a various data from the production, including but not limited to:
- telemetry data from the observed machines (vibration, voltage, temperature etc)
- errors and logs data relevant to each machine,
- failure data, when a certain component is replaced, etc
- quiality and accuracy data, machine properties, models, age etc.


### 3 Steps in Predictive Maintenance

Usually, every Predictive Maintenance technique should proceed by the following 3 main steps:
1. **Collect Data**  - collect all possible descriptions, historical and real-time data, usually by using IOT devices, various loggers, technical documentation, etc.
2. **Predict Failures** - collected data can be used and transformed into machine learning ready data sets, and build a machine learning model to predict the failures of the components in the set of machines in the production.
3. **React** - by obtaining the information which components will fail in the near future, we can activate the process of replacement so the component will be replaced before it fails, and the proidcution process will not be interrupted.

## Predict Failures

In this article, the second step will be presented, which will be related to data preparation.  In order to predict failures in the production process, a set of data transformations, cleaning, feature engineering, and selection must be performed to prepare the data for building a machine learning model.
The data preparation part plays a crucially step in the model building since a quality data preparation will directly reflect on the model accuracy and reliability.

## Software requirements 

In this article, the complete procedure in data preparation is presented. The whole process is performed using:

- `.NET Jupyter Notebook`- .NET implementation of popular Jupyer Notebook, 
- `ML.NET` - Microsoft open-source framework for Machine Learning on .NET Platform and 
- `Daany` - **DA**ta **AN**al**Y**tics library with the implementation of DataFrame, Time series decomposition and various statistical parameters. It can be found at Github, it is distributed as a Nuget package.

### The Step 1. Notebook preparation

In order to complete this task, we shoudl install several Nuget packages, and inlcude several using keywords.
The following code block shows the using stateent, as some data related to notebook formatting when the data is shown. 



**Note:** *nuget package installation must be in the first cell of the Notebook, otherwize the notebook will not work as expected. Hope this will change once the final version is released.*

In [2]:
//using Microsoft.ML.Data;
using XPlot.Plotly;
using System;
using System.Collections.Generic;
using System.Drawing;
using System.Linq;

//using statement of Daany package
using Daany;
using Daany.MathStuff;
using Daany.Ext;

//
using Microsoft.ML;

In [3]:
//DataFrame formatter
using Microsoft.AspNetCore.Html;
Formatter<DataFrame>.Register((df, writer) =>
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent) th(c)));
    
    //renders the rows
    var rows = new List<List<IHtmlContent>>();
    var take = 20;
    
    //
    for (var i = 0; i < Math.Min(take, df.RowCount()); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(df.Index[i]));
        foreach (var obj in df[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }
    
    var t = table(
        thead(
            headers),
        tbody(
            rows.Select(
                r => tr(r))));
    
    writer.Write(t);
}, "text/html");

### Download the data

In order to start with data preparation, we need data. The data can be found at Azure blob storage. The data is maintained by [Azure Gallery Article](https://gallery.azure.ai/Notebook/Predictive-Maintenance-Modelling-Guide-Python-Notebook-1). 

Once the data are downloaded from the blob storage, they will not be downloaded again, and they will be used local copies.

In [4]:
//URL Paths for the data files used in the Notebook
var urlTelemetry="https://azuremlsampleexperiments.blob.core.windows.net/datasets/PdM_telemetry.csv";
var urlErrors="https://azuremlsampleexperiments.blob.core.windows.net/datasets/PdM_errors.csv";
var urlMaintenance="https://azuremlsampleexperiments.blob.core.windows.net/datasets/PdM_maint.csv";
var urlFailures="https://azuremlsampleexperiments.blob.core.windows.net/datasets/PdM_failures.csv";
var urlMachines="https://azuremlsampleexperiments.blob.core.windows.net/datasets/PdM_machines.csv";

In [5]:
//check if the file exists
if(!System.IO.File.Exists("data/PdM_telemetry.csv"))
{
    using (System.Net.WebClient fileDownloader = new System.Net.WebClient())
    {
      fileDownloader.DownloadFile(urlTelemetry, "data/PdM_telemetry.csv");
    }
}
else 
   display("Telemetry file exist");

Telemetry file exist

In [6]:
//check if the file exists
if(!System.IO.File.Exists("data/PdM_errors.csv"))
{
    using (System.Net.WebClient fileDownloader = new System.Net.WebClient())
    {
      fileDownloader.DownloadFile(urlErrors, "data/PdM_errors.csv");
    }
}
else 
    display("Error file exist");

Error file exist

In [7]:
//check if the file exists
if(!System.IO.File.Exists("data/PdM_maint.csv"))
{
    using (System.Net.WebClient fileDownloader = new System.Net.WebClient())
    {
      fileDownloader.DownloadFile(urlMaintenance, "data/PdM_maint.csv");
    }
}
else 
    display("PdM_maint file exist");


PdM_maint file exist

In [8]:
//check if the file exists
if(!System.IO.File.Exists("data/PdM_failures.csv"))
{
    using (System.Net.WebClient fileDownloader = new System.Net.WebClient())
    {
      fileDownloader.DownloadFile(urlFailures, "data/PdM_failures.csv");
    }
}
else 
    display("PdM_failures file exist");

PdM_failures file exist

In [9]:

//check if the file exists
if(!System.IO.File.Exists("data/PdM_machines.csv"))
{
    using (System.Net.WebClient fileDownloader = new System.Net.WebClient())
    {
      fileDownloader.DownloadFile(urlMachines, "data/PdM_machines.csv");
    }
}
else 
    display("urlMachines file exist");

urlMachines file exist

Once we have files on the local disk, we can load them into application memory and create `Danny` `DataFrame` objects.

### The Data

The data we are using for predictive maintenance can be classified to:
- telemetry - which collects historical data about machine behavior (voltage, vibration, etc)
- errors - the data about warnings and errors in the machines
- maint - data about replacement and maintenance for the machines,
- machines - descriptive information about the machines,
- failures - data when a certain machine is stopped, due to component failure.

We load all the files in order to fully prepare data for the training process. The following code sample, loads the data in to application memory.

In [10]:
%%time
//Load ALL 5 data frame files
//DataFrame Cols: datetime,machineID,volt,rotate,pressure,vibration
var telemetry = DataFrame.FromCsv("data/PdM_telemetry.csv", dformat: "yyyy-mm-dd hh:mm:ss");
var errors = DataFrame.FromCsv("data/PdM_errors.csv", dformat: "yyyy-mm-dd hh:mm:ss");
var maint = DataFrame.FromCsv("data/PdM_maint.csv", dformat: "yyyy-mm-dd hh:mm:ss");
var failures = DataFrame.FromCsv("data/PdM_failures.csv", dformat: "yyyy-mm-dd hh:mm:ss");
var machines = DataFrame.FromCsv("data/PdM_machines.csv", dformat: "yyyy-mm-dd hh:mm:ss");

Wall time: 8082.7142ms

In [11]:
display($"DataFrame: {nameof(telemetry)} size is ({telemetry.RowCount()},{telemetry.ColCount()})");
display($"DataFrame: {nameof(errors)} size is ({errors.RowCount()},{errors.ColCount()})");
display($"DataFrame: {nameof(maint)} size is ({maint.RowCount()},{maint.ColCount()})");
display($"DataFrame: {nameof(failures)} size is ({failures.RowCount()},{failures.ColCount()})");
display($"DataFrame: {nameof(machines)} size is ({machines.RowCount()},{machines.ColCount()})");

DataFrame: telemetry size is (876100,6)

DataFrame: errors size is (3919,3)

DataFrame: maint size is (3286,3)

DataFrame: failures size is (761,3)

DataFrame: machines size is (100,3)

As can be seen, the main telemetry data contains nearl 900 000 rows, which is enought to trin a ML model.

## Telemetry

The first data source is the telemetry data about machines. It consists of `voltage`, `rotation`, `pressure`, and `vibration` measurements measured from 100 machines in real-time hourly. The time period the data has been collected is during the year 2015. 
The following data shows the first 10 records in the dataset.

In [12]:
telemetry.Head(10)

0,2015-01-01 06:00:00Z,1,176.21785,418.5041,113.077934,45.087685
1,2015-01-01 07:00:00Z,1,162.87923,402.7475,95.460526,43.41397
2,2015-01-01 08:00:00Z,1,170.9899,527.34985,75.23791,34.17885
3,2015-01-01 09:00:00Z,1,162.46283,346.14932,109.24856,41.122143
4,2015-01-01 10:00:00Z,1,157.61002,435.37686,111.88665,25.99051
5,2015-01-01 11:00:00Z,1,172.50484,430.32336,95.92704,35.655018
6,2015-01-01 12:00:00Z,1,156.55603,499.07162,111.755684,42.75392
7,2015-01-01 13:00:00Z,1,172.52278,409.62473,101.00108,35.48201
8,2015-01-01 14:00:00Z,1,175.32452,398.64877,110.62436,45.48229
9,2015-01-01 15:00:00Z,1,169.21843,460.85068,104.84823,39.901737


A description of the whole dataset is shown on the next cell. As can be seen, we have nearly million records for the machines, which is good starting point for the analysis.

In [13]:
telemetry.Describe()

Count,876100,876100,876100,876100,876100
Unique,100,778473,814048,802996,802761
Top,1,162.570465,465.84726,100.059151,41.085587
Freq,8761,5,5,7,5
Mean,50.5,170.77774,446.605133,100.858665,40.385006
Std,28.866087,15.509114,52.673886,11.048679,5.370361
Min,1,97.333603,138.432068,51.237106,14.877054
25%,25.75,160.304928,412.305717,93.498182,36.7773
Median,50.5,170.60733,447.558167,100.425552,40.237247
75%,75.25,181.00449,482.17659,107.555229,43.784938
Max,100,255.12471,695.020996,185.951996,76.791069


In case we want to see the visualization of the telemetry data, we can select on of several column and show it. 

In [59]:
//define filters columns (two dates columns)
string[] cols = new string[] { "datetime", "datetime", "machineID" };

//filter values for each column
var valls = new List<object>();
valls.Add(new DateTime(2015, 1, 1)); valls.Add(new DateTime(2015, 2, 1)); valls.Add(1);

//now filter operator 
FilterOperator[] oprs = new FilterOperator[] { FilterOperator.Greather, FilterOperator.LessOrEqual, FilterOperator.Equal };

//perform filtering 
var machine1Df = telemetry.Filter(cols, valls.ToArray(), oprs);

//plot coordinates
var x = machine1Df["datetime"].Select(t => Convert.ToDateTime(t)).ToArray();
var voltage1 = machine1Df["volt"].Select(t => Convert.ToDouble(t)).ToArray();

//Plot Telemetry data
var chart = Chart.Plot(
    new Graph.Scattergl()
    {
        x = x,
        y = voltage1,
      //  mode = "markers",  
    }
    
);
//
var layout = new XPlot.Plotly.Layout.Layout() 
    { title = "Machine Voltage",
     xaxis=new XPlot.Plotly.Graph.Xaxis() { title="Time" }, 
     yaxis = new XPlot.Plotly.Graph.Yaxis() { title = "Voltage" } };
//put layout into chart
chart.WithLayout(layout);
display(chart)

## Errors
One of the most important information in every Predictive Maintenance system is Error data. Actually errors are non-breaking recorded events while the machine is still operational. The error date and times are rounded to the closest hour since the telemetry data is collected at an hourly rate.  

In [15]:
errors.Head()

0,2015-01-03 07:00:00Z,1,error1
1,2015-01-03 20:00:00Z,1,error3
2,2015-01-04 06:00:00Z,1,error5
3,2015-01-10 15:00:00Z,1,error4
4,2015-01-22 10:00:00Z,1,error4


In [16]:
//count number of errors 
var barValue = errors["errorID"].GroupBy(v => v)
        .OrderBy(group => group.Key)
        .Select(group => Tuple.Create(group.Key, group.Count()));

//Plot Errors data
var chart = Chart.Plot(
    new Graph.Bar()
    {
       x = barValue.Select(x=>x.Item1),
       y = barValue.Select(x=>x.Item2),
      //  mode = "markers",  
    }
    
);
var layout = new XPlot.Plotly.Layout.Layout() 
    { title = "Error distribution",
     xaxis=new XPlot.Plotly.Graph.Xaxis() { title="Error name" }, 
     yaxis = new XPlot.Plotly.Graph.Yaxis() { title = "Error Count" } };
//put layout into chart
chart.WithLayout(layout);

display(chart)


## Maintenance

The Maintenance is the next PrM component which tells us about scheduled and unscheduled maintenance. The maintenance contains the records which correspond to both regular inspection of components as well as failures. To add the record into the maintenance table a component must be replaced during the scheduled inspection or replaced due to a breakdown. In case the records are created due to breakdowns are called `failures`. Maintenance contains the data from 2014 and 2015 years.

In [17]:
maint.Head()

0,2014-06-01 06:00:00Z,1,comp2
1,2014-07-16 06:00:00Z,1,comp4
2,2014-07-31 06:00:00Z,1,comp3
3,2014-12-13 06:00:00Z,1,comp1
4,2015-01-05 06:00:00Z,1,comp4


In [18]:
//count number of errors 
var barValue = maint["comp"].GroupBy(v => v)
        .OrderBy(group => group.Key)
        .Select(group => Tuple.Create(group.Key, group.Count()));

//Plot Errors data
var chart = Chart.Plot(
    new Graph.Bar()
    {
       x = barValue.Select(x=>x.Item1),
       y = barValue.Select(x=>x.Item2),
      //  mode = "markers",  
    }
    
);
var layout = new XPlot.Plotly.Layout.Layout() 
    { title = "Components Replacements",
     xaxis=new XPlot.Plotly.Graph.Xaxis() { title="Component Name" }, 
     yaxis = new XPlot.Plotly.Graph.Yaxis() { title = "Number of components replaced" } };
//put layout into chart
chart.WithLayout(layout);

display(chart)

## Machines

The data include information about 100 machines which are subject of the Predictive Maintenance analysis. The information includes: `model type`, and machine `age`.

In [19]:
machines.Head()

0,1,model3,18
1,2,model4,7
2,3,model3,8
3,4,model3,7
4,5,model3,2


In [20]:
//count number of errors 
var barValue = machines["age"].GroupBy(v => v)
        .OrderBy(group => group.Key)
        .Select(group => Tuple.Create(group.Key, group.Count()));

//Plot machine data
var chart = Chart.Plot(
    new Graph.Bar()
    {
       x = barValue.Select(x=>x.Item1),
       y = barValue.Select(x=>x.Item2),
       name = "model1",  
    }
    
);
var layout = new XPlot.Plotly.Layout.Layout() 
    { title = "Components Replacements",
     xaxis=new XPlot.Plotly.Graph.Xaxis() { title="Machine Age" }, 
     yaxis = new XPlot.Plotly.Graph.Yaxis() { title = "Count" } };
//put layout into chart
chart.WithLayout(layout);

display(chart)


In [21]:
//Distribution of models across age
var d1 = machines.Filter("model", "model1", FilterOperator.Equal)["age"]
                                    .GroupBy(g => g).Select(g=>(g.Key,g.Count()));
var d2 = machines.Filter("model", "model2", FilterOperator.Equal)["age"]
                                    .GroupBy(g => g).Select(g=>(g.Key,g.Count()));
var d3 = machines.Filter("model", "model3", FilterOperator.Equal)["age"]
                                    .GroupBy(g => g).Select(g=>(g.Key,g.Count()));
var d4 = machines.Filter("model", "model4", FilterOperator.Equal)["age"]
                                    .GroupBy(g => g).Select(g=>(g.Key,g.Count()));
//define bars
var b1 = new Graph.Bar(){ x = d1.Select(x=>x.Item1),y = d1.Select(x=>x.Item2),name = "model1"};
var b2 = new Graph.Bar(){ x = d2.Select(x=>x.Item1),y = d2.Select(x=>x.Item2),name = "model2"};
var b3 = new Graph.Bar(){ x = d3.Select(x=>x.Item1),y = d3.Select(x=>x.Item2),name = "model3"};
var b4 = new Graph.Bar(){ x = d4.Select(x=>x.Item1),y = d4.Select(x=>x.Item2),name = "model4"};
    
    
//Plot machine data
var chart = Chart.Plot(new[] {b1,b2,b3,b4});

var layout = new XPlot.Plotly.Layout.Layout() 
    { title = "Components Replacements",barmode="stack",
     xaxis=new XPlot.Plotly.Graph.Xaxis() { title="Machine Age" }, 
     yaxis = new XPlot.Plotly.Graph.Yaxis() { title = "Count" } };
//put layout into chart
chart.WithLayout(layout);

display(chart)

## Failures

The Failures represent the replacements of the components due to the failure of the machines. Once the failure is happend the machine is stopped. This is a crucial difference between Errors and Failures.

In [22]:
failures.Head()

0,2015-01-05 06:00:00Z,1,comp4
1,2015-03-06 06:00:00Z,1,comp1
2,2015-04-20 06:00:00Z,1,comp2
3,2015-06-19 06:00:00Z,1,comp4
4,2015-09-02 06:00:00Z,1,comp4


In [60]:
//count number of failures  
var falValues = failures["failure"].GroupBy(v => v)
        .OrderBy(group => group.Key)
        .Select(group => Tuple.Create(group.Key, group.Count()));

//Plot Failure data
var chart = Chart.Plot(
    new Graph.Bar()
    {
       x = falValues.Select(x=>x.Item1),
       y = falValues.Select(x=>x.Item2),
      //  mode = "markers",  
    }
    
);
var layout = new XPlot.Plotly.Layout.Layout() 
    { title = "Failure Distribution acorss machines",
     xaxis=new XPlot.Plotly.Graph.Xaxis() { title="Component Name" }, 
     yaxis = new XPlot.Plotly.Graph.Yaxis() { title = "Number of components replaces" } };
//put layout into chart
chart.WithLayout(layout);

display(chart)

## Feature Engineering

This section contains several feature engineering methods used to create features based on the machines' properties.

### Lagged Telemetry Features

First, we are going to create several lagged telemetry data, since telemetry data are classic time series data.

In the following, the rolling mean and standard deviation of the telemetry data over the last 3-hour lag window is calculated for every 3 hours.

In [24]:
//prepare rolling aggregation for each column for average values
var agg_curent = new Dictionary<string, Aggregation>()
 {
    { "datetime", Aggregation.Last }, { "volt", Aggregation.Last }, { "rotate", Aggregation.Last },
    { "pressure", Aggregation.Last },{ "vibration", Aggregation.Last }
  };
//prepare rolling aggregation for each column for average values
var agg_mean = new Dictionary<string, Aggregation>()
 {
    { "datetime", Aggregation.Last }, { "volt", Aggregation.Avg }, { "rotate", Aggregation.Avg },
    { "pressure", Aggregation.Avg },{ "vibration", Aggregation.Avg }
  };
//prepare rolling aggregation for each column for std values
var agg_std = new Dictionary<string, Aggregation>()
{
   { "datetime", Aggregation.Last }, { "volt", Aggregation.Std }, { "rotate", Aggregation.Std },
    { "pressure", Aggregation.Std },{ "vibration", Aggregation.Std }
};

In [25]:
//group Telemetry data by machine ID
var groupedTelemetry = telemetry.GroupBy("machineID");

In [26]:
//calculate rolling mean for grouped data for each 3 hours
var _3AvgValue = groupedTelemetry.Rolling(3, 3, agg_mean)
                 .Create(("machineID", null), ("datetime", null),("volt", "voltmean_3hrs"), ("rotate", "rotatemean_3hrs"),
                         ("pressure", "pressuremean_3hrs"), ("vibration", "vibrationmean_3hrs"));
//show head of the newely generated table
_3AvgValue.Head()

0,1,2015-01-01 08:00:00Z,170.028992,449.533813,94.592125,40.893501
1,1,2015-01-01 11:00:00Z,164.192566,403.94986,105.687416,34.25589
2,1,2015-01-01 14:00:00Z,168.134445,435.781708,107.793709,41.239407
3,1,2015-01-01 17:00:00Z,165.51445,430.472839,101.703285,40.373741
4,1,2015-01-01 20:00:00Z,168.809341,437.111115,90.911064,41.738544


In [27]:
_3AvgValue.Tail()

291995,100,2015-12-31 17:00:00Z,177.758759,458.391632,103.183159,46.846592
291996,100,2015-12-31 20:00:00Z,166.078949,424.431183,98.344414,30.907459
291997,100,2015-12-31 23:00:00Z,158.327988,504.056274,108.055443,40.109707
291998,100,2016-01-01 02:00:00Z,170.306961,438.653625,102.34623,47.481232
291999,100,2016-01-01 05:00:00Z,182.525421,435.932068,98.798172,42.757629


In [28]:
//calculate rolling std for grouped datat fro each 3 hours
var _3StdValue = groupedTelemetry.Rolling(3, 3, agg_mean)
                 .Create(("machineID", null), ("datetime", null),("volt", "voltsd_3hrs"), ("rotate", "rotatesd_3hrs"),
                         ("pressure", "pressuresd_3hrs"), ("vibration", "vibrationsd_3hrs"));
//show head of the newely generated table
_3StdValue.Head()

0,1,2015-01-01 08:00:00Z,170.028992,449.533813,94.592125,40.893501
1,1,2015-01-01 11:00:00Z,164.192566,403.94986,105.687416,34.25589
2,1,2015-01-01 14:00:00Z,168.134445,435.781708,107.793709,41.239407
3,1,2015-01-01 17:00:00Z,165.51445,430.472839,101.703285,40.373741
4,1,2015-01-01 20:00:00Z,168.809341,437.111115,90.911064,41.738544


For caputing a longer term effect 24 hours lag features we are going to calculate rolling avf and std.

In [30]:
//calculate rolling avg and std for each 24 hours
var _24AvgValue = groupedTelemetry.Rolling(24, 3, agg_mean)
                .Create(("machineID", null), ("datetime", null),
                        ("volt", "voltmean_24hrs"), ("rotate", "rotatemean_24hrs"),
                        ("pressure", "pressuremean_24hrs"), ("vibration", "vibrationmean_24hrs"));
var _24StdValue = groupedTelemetry.Rolling(24, 3, agg_std)
                .Create(("machineID", null), ("datetime", null),
                        ("volt", "voltsd_24hrs"), ("rotate", "rotatesd_24hrs"),
                        ("pressure", "pressuresd_24hrs"), ("vibration", "vibrationsd_24hrs"));

### Merging telemetry features

Once we have rolling lag features calculated, we can merge them into one data frame:

In [31]:
//before merge all features create set of features from the current values for every 3 or 24 hours
DataFrame _1CurrentValue = groupedTelemetry.Rolling(3, 3, agg_curent)
                            .Create(("machineID", null), ("datetime", null),
                            ("volt", null), ("rotate", null), ("pressure", null), ("vibration", null));

In [32]:
//merge all telemetry data frames into one
var mergeCols= new string[] { "machineID", "datetime" };
var df1 = _1CurrentValue.Merge(_3AvgValue, mergeCols, mergeCols, JoinType.Left, suffix: "df1");   
                                            
var df2 = df1.Merge(_24AvgValue, mergeCols, mergeCols, JoinType.Left, suffix: "df2");
                                 
var df3 = df2.Merge(_3StdValue, mergeCols, mergeCols, JoinType.Left, suffix: "df3");
                                
var df4 = df3.Merge(_24StdValue, mergeCols, mergeCols, JoinType.Left, suffix: "df4");

In [33]:
//select final dataset for the telemetry
var telDF = df4["machineID","datetime","volt","rotate", "pressure", "vibration",
                 "voltmean_3hrs","rotatemean_3hrs","pressuremean_3hrs","vibrationmean_3hrs",
                 "voltmean_24hrs","rotatemean_24hrs","pressuremean_24hrs","vibrationmean_24hrs",
                 "voltsd_3hrs", "rotatesd_3hrs","pressuresd_3hrs","vibrationsd_3hrs",
                 "voltsd_24hrs", "rotatesd_24hrs","pressuresd_24hrs","vibrationsd_24hrs"];

//remove NANs
var telemetry_final = telDF.DropNA();

In [34]:
telemetry_final.Head()

0,1,2015-01-02 05:00:00Z,181.257523,495.777954,93.439293,52.355877,180.133789,440.608337,94.13797,41.551544,169.73381,445.179871,96.797112,40.385159,180.133789,440.608337,94.13797,41.551544,11.233122,48.717397,10.079879,5.853208
1,1,2015-01-02 08:00:00Z,160.528854,486.459045,86.944275,42.992508,176.364288,439.34964,101.553207,36.105579,170.525726,443.90686,97.667252,39.786671,176.364288,439.34964,101.553207,36.105579,12.59195,46.930281,9.406794,6.098173
2,1,2015-01-02 11:00:00Z,173.394516,439.579468,97.675827,37.117104,160.384567,424.385315,99.598724,36.094639,170.049713,446.461273,96.906158,40.016514,160.384567,424.385315,99.598724,36.094639,13.277338,42.836778,9.071471,5.481724
3,1,2015-01-02 14:00:00Z,152.420776,497.840607,101.641693,30.665184,170.472458,442.93399,102.380585,40.483002,170.341965,447.355316,96.229523,39.921963,170.472458,442.93399,102.380585,40.483002,13.81716,42.808631,8.256793,5.862312
4,1,2015-01-02 17:00:00Z,164.512222,424.542633,102.485527,42.243374,163.263809,468.937561,102.726646,40.921803,170.060638,452.163391,96.357445,39.990471,163.263809,468.937561,102.726646,40.921803,14.79287,42.52529,8.669605,5.907157


## Lag Features from Errors

Unlike telemetry that had numerical values, errors have categorical values denoting the type of error that occurred at a time-stamp. We are going to aggregate categories of the error with different types of errors that occurred in the lag window.

First, encode the errors with One-Hot-Encoding:

In [35]:
var mlContext = new MLContext(seed:2019);
//One Hot Encoding of error column
var encodedErr = errors.EncodeColumn(mlContext, "errorID");

//sum duplicated erros by machine and date
var errors_aggs = new Dictionary<string, Aggregation>();
errors_aggs.Add("error1", Aggregation.Sum);
errors_aggs.Add("error2", Aggregation.Sum);
errors_aggs.Add("error3", Aggregation.Sum);
errors_aggs.Add("error4", Aggregation.Sum);
errors_aggs.Add("error5", Aggregation.Sum);

//group and sum duplicated errors
encodedErr =  encodedErr.GroupBy(new string[] { "machineID", "datetime" }).Aggregate(errors_aggs);

//
encodedErr = encodedErr.Create(("machineID", null), ("datetime", null),
                        ("error1", "error1sum"), ("error2", "error2sum"),
                        ("error3", "error3sum"), ("error4", "error4sum"), ("error5", "error5sum"));
encodedErr.Head()                       

0,1,2015-01-03 07:00:00Z,1,0,0,0,0
1,1,2015-01-03 20:00:00Z,0,0,1,0,0
2,1,2015-01-04 06:00:00Z,0,0,0,0,1
3,1,2015-01-10 15:00:00Z,0,0,0,1,0
4,1,2015-01-22 10:00:00Z,0,0,0,1,0


In [36]:
// align errors with telemetry datetime values so that we can calculate aggregations
var er = telemetry.Merge(encodedErr,mergeCols, mergeCols, JoinType.Left, suffix: "error");
//
er = er["machineID","datetime", "error1sum", "error2sum", "error3sum", "error4sum", "error5sum"];
//fill missing values with 0
er.FillNA(0);
er.Head()   

0,1,2015-01-01 06:00:00Z,0,0,0,0,0
1,1,2015-01-01 07:00:00Z,0,0,0,0,0
2,1,2015-01-01 08:00:00Z,0,0,0,0,0
3,1,2015-01-01 09:00:00Z,0,0,0,0,0
4,1,2015-01-01 10:00:00Z,0,0,0,0,0


In [37]:
//count the number of errors of different types in the last 24 hours, for every 3 hours
//define aggregation
var errors_aggs1 = new Dictionary<string, Aggregation>()
{
  { "datetime", Aggregation.Last },{ "error1sum", Aggregation.Sum }, { "error2sum", Aggregation.Sum }, 
  { "error3sum", Aggregation.Sum },{ "error4sum", Aggregation.Sum },
  { "error5sum", Aggregation.Sum }
};

//count the number of errors of different types in the last 24 hours,  for every 3 hours
var eDF = er.GroupBy(new string[] { "machineID"}).Rolling(24, 3, errors_aggs1);

//
var newdf=  eDF.DropNA();

var errors_final = newdf.Create(("machineID", null), ("datetime", null),
                        ("error1sum", "error1count"), ("error2sum", "error2count"),
                        ("error3sum", "error3count"), ("error4sum", "error4count"), ("error5sum", "error5count"));
errors_final.Head()

0,1,2015-01-02 05:00:00Z,0,0,0,0,0
1,1,2015-01-02 08:00:00Z,0,0,0,0,0
2,1,2015-01-02 11:00:00Z,0,0,0,0,0
3,1,2015-01-02 14:00:00Z,0,0,0,0,0
4,1,2015-01-02 17:00:00Z,0,0,0,0,0


## Time Since Last Replacement


As the main task here is how to create a relevant feature in order to create a quality data set for the machine learning part. One of the good features would be the number of replacements of each component in the last 3 months to incorporate the frequency of replacements. 

Furthermore, we can calculate how long it has been since a component is last replaced as that would be expected to correlate better with component failures since the longer a component is used, the more degradation should be expected.
 As first we are going to encode the maintenance table:

In [38]:
//One Hot Encoding of error column
var encMaint = maint.EncodeColumn(mlContext, "comp");
encMaint.Head()

0,2014-06-01 06:00:00Z,1,comp2,1,0,0,0
1,2014-07-16 06:00:00Z,1,comp4,0,1,0,0
2,2014-07-31 06:00:00Z,1,comp3,0,0,1,0
3,2014-12-13 06:00:00Z,1,comp1,0,0,0,1
4,2015-01-05 06:00:00Z,1,comp4,0,1,0,0


In [39]:
//create separate data frames in order to calculate proper time since last replacement 
DataFrame dfComp1 = encMaint.Filter("comp1", 1, FilterOperator.Equal)["machineID", "datetime"];
DataFrame dfComp2 = encMaint.Filter("comp2", 1, FilterOperator.Equal)["machineID", "datetime"];;
DataFrame dfComp3 = encMaint.Filter("comp3", 1, FilterOperator.Equal)["machineID", "datetime"];;
DataFrame dfComp4 = encMaint.Filter("comp4", 1, FilterOperator.Equal)["machineID", "datetime"];;

dfComp4.Head()

0,1,2014-07-16 06:00:00Z
1,1,2015-01-05 06:00:00Z
2,1,2015-02-04 06:00:00Z
3,1,2015-06-19 06:00:00Z
4,1,2015-09-02 06:00:00Z


In [40]:
//from telemetry data create helped dataframe so we can calculate additional column from the maintenance data frame
var compData = telemetry_final.Create(("machineID", null), ("datetime", null));

In [41]:
%%time
//calculate new set of columns so that we have information the time since last replacement of each component separetly
var newCols= new string[]{"sincelastcomp1","sincelastcomp2","sincelastcomp3","sincelastcomp4"};
var calcValues= new object[4];

//perform calculation
compData.AddCalculatedColumns(newCols,(row, i)=>
{
    var machineId = Convert.ToInt32(row["machineID"]);
    var date = Convert.ToDateTime(row["datetime"]);
    
    var maxDate1 = dfComp1.Filter("machineID", machineId, FilterOperator.Equal)["datetime"]
        .Where(x => (DateTime)x <= date).Select(x=>(DateTime)x).Max();
    var maxDate2 = dfComp2.Filter("machineID", machineId, FilterOperator.Equal)["datetime"]
        .Where(x => (DateTime)x <= date).Select(x=>(DateTime)x).Max();
    var maxDate3 = dfComp3.Filter("machineID", machineId, FilterOperator.Equal)["datetime"]
        .Where(x => (DateTime)x <= date).Select(x=>(DateTime)x).Max();
    var maxDate4 = dfComp4.Filter("machineID", machineId, FilterOperator.Equal)["datetime"]
        .Where(x => (DateTime)x <= date).Select(x=>(DateTime)x).Max();
        
    //perform calculation
    calcValues[0] = (date - maxDate1).TotalDays;
    calcValues[1] = (date - maxDate2).TotalDays;
    calcValues[2] = (date - maxDate3).TotalDays;
    calcValues[3] = (date - maxDate4).TotalDays;
    return calcValues;
});

Wall time: 178708.9764ms

In [42]:
var maintenance_final = compData;
maintenance_final.Head()

0,1,2015-01-02 05:00:00Z,19.958333333333332,214.95833333333334,154.95833333333334,169.95833333333334
1,1,2015-01-02 08:00:00Z,20.083333333333332,215.08333333333334,155.08333333333334,170.08333333333334
2,1,2015-01-02 11:00:00Z,20.208333333333332,215.20833333333334,155.20833333333334,170.20833333333334
3,1,2015-01-02 14:00:00Z,20.333333333333332,215.33333333333334,155.33333333333334,170.33333333333334
4,1,2015-01-02 17:00:00Z,20.458333333333332,215.45833333333334,155.45833333333334,170.45833333333334


## Machine Features

The machine data set contains descriptive information about machines like the type of machines and their ages which is the years in service. 

In [43]:
machines.Head()

0,1,model3,18
1,2,model4,7
2,3,model3,8
3,4,model3,7
4,5,model3,2


## Joining features into final ML ready data set

As the last step in Feature engineering, we are performing merging all features into one data set. 

In [44]:
var merge2Cols=new string[]{"machineID"};
var fdf1= telemetry_final.Merge(errors_final, mergeCols, mergeCols,JoinType.Left, suffix: "er");
var fdf2 = fdf1.Merge(maintenance_final, mergeCols,mergeCols,JoinType.Left, suffix: "mn");
var features_final = fdf2.Merge(machines, merge2Cols,merge2Cols,JoinType.Left, suffix: "ma");

In [62]:
features_final= features_final["datetime", "machineID", 
            "voltmean_3hrs", "rotatemean_3hrs", "pressuremean_3hrs", "vibrationmean_3hrs",
            "voltsd_3hrs", "rotatesd_3hrs", "pressuresd_3hrs", "vibrationsd_3hrs", 
            "voltmean_24hrs", "rotatemean_24hrs", "pressuremean_24hrs", "vibrationmean_24hrs", 
            "voltsd_24hrs","rotatesd_24hrs", "pressuresd_24hrs", "vibrationsd_24hrs", 
            "error1count", "error2count", "error3count", "error4count", "error5count", 
            "sincelastcomp1", "sincelastcomp2", "sincelastcomp3", "sincelastcomp4", 
            "model", "age"];
//

features_final.Head();
DataFrame.ToCsv("data/final_features.csv", features_final);

# Define Label Column


The Label in prediction maintenance should be the probability that a machine will fail in the near future due to a failure certain component. If we take 24 hours to be a task for this problem, the label construction is consists of a new column in the feature data set which indicate if certain machine will fail or not in the next 24 hours due to failure one of several components. 

With this way we are defining the label as a categorical variable containing:
- `none` - if the machine will not fail in the next 24 hours,
- `comp1` to `comp4` - if the machine will fail in the next 24 hours due to the failure of cetain components.

Since we can experiment with the label construction by applying different conditions, we can implement methods that take several arguments in order to define the general problem.

In [46]:
failures.Describe(false)

Count,761,761,761
Unique,302,98,4
Top,2015-01-02 03:00:00Z,99,comp2
Freq,18,19,259
Mean,<null>,51.911958,<null>
Std,<null>,29.515542,<null>
Min,2015-01-02 03:00:00Z,1,<null>
25%,<null>,24,<null>
Median,<null>,51,<null>
75%,<null>,79,<null>
Max,2015-12-31 06:00:00Z,100,<null>


In [55]:
//constructing the label column which indicate if the current machine will 
//fail in the next `predTime` (24 hours as default) due to failur certain component.
//create final data frame from from feature df
var finalDf = new DataFrame(features_final);

//group failures by machineID and datetime 
string[] cols = new string[] {  "machineID" , "datetime"};
var failDfgrp = failures.GroupBy(cols);

//Add failure column to  finalDF
var rV = new object[] { "none" };
finalDf.AddCalculatedColumns(new string[]{"failure"}, (object[] row, int i) => rV);

//create new data frame from featuresDF by grouping machineID and datatime
var featureDfGrouped = finalDf["datetime","machineID", "failure"].GroupBy(cols);

//now look for every failure and calculate if the machine will fail in the last 24 hours
//in case two or more components were failed for the ssame machine add new row in df
var failureDfExt = featureDfGrouped.Transform((xdf) =>
{
    //extract the row from featureDfGrouped
    var xdfRow = xdf[0].ToList();
    var refDate = (DateTime)xdfRow[0];
    var machineID = (int)xdfRow[1];

    //now look if the failure contains the machineID
    if(failDfgrp.Group2.ContainsKey(machineID))
    {
        //get the date and calculate total hours
        var dff = failDfgrp.Group2[machineID];

        foreach (var dfff in dff)
        {
            for (int i = 0; i < dfff.Value.RowCount(); i++)
            {
                //"datetime","machineID","failure"
                var frow = dfff.Value[i].ToList();
                var dft = (DateTime)frow[0];
                
                //if total hours is less or equal than 24 hours set component to the failure column
                var totHours = (dft - refDate).TotalHours;
                if (totHours <= 24 && totHours >=0)
                {
                    if (xdf.RowCount() > i)
                        xdf["failure", i] = frow[2];
                    else//in case two components were failed for the same machine and 
                        //at the same time, add new row with new component name
                    {
                        var r = xdf[0].ToList();
                        r[2] = frow[2];
                        xdf.AddRow(r);
                    }
                }
            }
        }
    }
    return xdf;
});

//Now merge extended failure Df with featureDF
var final_dataframe = finalDf.Merge(failureDfExt, cols, cols,JoinType.Left, "fail");

//define final set of columns
final_dataframe = final_dataframe["datetime", "machineID",
"voltmean_3hrs", "rotatemean_3hrs", "pressuremean_3hrs", "vibrationmean_3hrs",
"voltsd_3hrs", "rotatesd_3hrs", "pressuresd_3hrs", "vibrationsd_3hrs",
"voltmean_24hrs", "rotatemean_24hrs", "pressuremean_24hrs", "vibrationmean_24hrs",
"voltsd_24hrs", "rotatesd_24hrs", "pressuresd_24hrs", "vibrationsd_24hrs",
"error1count", "error2count", "error3count", "error4count", "error5count",
"sincelastcomp1", "sincelastcomp2", "sincelastcomp3", "sincelastcomp4",
"model", "age", "failure_fail"];

//rename column
final_dataframe.Rename(("failure_fail", "failure"));

In [56]:
//save the file data frame to disk
DataFrame.ToCsv("data/final_dataFrame.csv",final_dataframe);

### Final Data Frame 

Lets see how looks like our final data frame. The final data frame contains 24 columns. Most of the columns are numerical. The `Model` column is categorical and it should be encoded once we prepare the macihne leraning part.

Also the lable column `failure` is categorical column containing 5 diferent categories: `none`, `comp1`, `comp2`, `comp3` and `comp4`. We can also see the data set is not balance, since we have `2785705` `none` and the rest of the rows in total of 5923 other categories. This is typical unbalanced dataset, and we should be carefull when evaluation models, because the mdoel which returns always `none` value will have more than 97% of accuracy.


In [58]:
final_dataframe.Describe(false)

Count,291628,291628,291628,291628,291628,291628,291628,291628,291628,291628,291628,291628,291628,291628,291628,291628,291628,291628,291628,291628,291628,291628,291628,291628,291628,291628,291628,291628,291628,291628
Unique,2913,100,272308,279828,277136,277107,272308,279828,277136,277107,243804,261603,255030,255013,286047,289707,283633,276144,6,5,5,6,6,3936,2792,2968,3160,4,20,5
Top,2015-01-23 08:00:00Z,13,174.906036,471.315857,96.265266,39.821896,174.906036,471.315857,96.265266,39.821896,169.303436,454.289856,99.549225,40.315128,13.861938,51.3554,9.490836,4.619621,0,0,0,0,0,0.958333,0.958333,0.958333,0.958333,model3,14,none
Freq,102,2937,6,5,4,4,6,5,4,4,6,5,6,5,4,3,4,4,275096,275094,276211,277243,279964,702,761,706,710,102027,40894,285705
Mean,<null>,50.493951,170.781545,446.553984,100.864825,40.390508,170.781545,446.553984,100.864825,40.390508,170.782456,446.554069,100.863917,40.390843,14.918931,49.949058,10.046866,5.002062,0.028166,0.027796,0.023626,0.020221,0.0104,53.497724,51.456651,52.631449,53.75702,<null>,11.332564,<null>
Std,<null>,28.869339,9.508674,33.193947,7.428436,3.485387,9.508674,33.193947,7.428436,3.485387,4.740511,18.208245,4.764441,2.074927,2.260515,7.690423,1.71209,0.799436,0.167834,0.166522,0.153587,0.141896,0.102057,62.474817,59.169589,58.774463,59.634162,<null>,5.826877,<null>
Min,2015-01-02 05:00:00Z,1,125.532501,211.811188,72.118637,26.569635,125.532501,211.811188,72.118637,26.569635,156.284729,267.008148,90.352638,35.253277,6.502827,19.83952,4.433239,2.107598,0,0,0,0,0,0.083333,0.083333,0.083333,0.083333,<null>,0,<null>
25%,<null>,25,164.449108,427.519211,96.239675,38.148529,164.449108,427.519211,96.239675,38.148529,168.06969,441.551948,98.670032,39.357998,13.358238,44.671207,8.92455,4.461183,0,0,0,0,0,13.333333,12.208333,13.208333,13.083333,<null>,7,<null>
Median,<null>,50,170.436142,448.353104,100.235771,40.149202,170.436142,448.353104,100.235771,40.149202,170.211388,449.187821,100.09861,40.071182,14.856054,49.615941,9.922136,4.958264,0,0,0,0,0,32.833333,29.708333,32.208333,32.458333,<null>,12,<null>
75%,<null>,75,176.615345,468.427635,104.408676,42.231729,176.615345,468.427635,104.408676,42.231729,172.457306,456.365272,101.615353,40.833646,16.396245,54.803683,10.981052,5.484203,0,0,0,0,0,68.583333,66.333333,67.208333,70.208333,<null>,16,<null>
Max,2016-01-01 05:00:00Z,100,241.420715,586.682861,162.309662,69.311325,241.420715,586.682861,162.309662,69.311325,220.569061,499.288788,152.660904,61.85284,27.914052,105.326447,28.867737,12.609035,2,2,2,2,2,491.958333,348.958333,370.958333,394.958333,<null>,20,<null>


In the next part, we are going to implement the training and evaluation process of the Predictive Maintenance model.